In [1]:
import win32com.client

## 1) 종목 코드 가져오기

In [2]:
instCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
print(instCpCybos.IsConnect) # 1이면 성공

1


증권사가 제공하는 API를 보면 일반적으로 종목 코드를 통해 특정 종목에 대한 시세를 조회하거나 매수/매도 주문을 넣을 수 있습니다. 따라서 API를 사용해 각 종목에 대한 종목 코드를 구하는 것은 가장 기본적인 작업이라 할 수 있습니다.

이번 절에서는 유가증권시장에 상장된 종목의 종목 코드와 종목명을 파이썬의 딕셔너리 자료구조에 저장하는 프로그램을 작성해보겠습니다.

그림 10.16과 같이 CybosPlus 도움말 파일에서 검색 탭을 이용해 CpCodeMgr 클래스를 찾습니다. CpCodeMgr 클래스는 각종 코드 정보 및 코드 목록을 구하는 데 사용합니다.

CpCodeMgr 클래스에서는 여러 메서드를 제공하는데 그중 GetStockListByMarket 메서드는 시장 구분에 따라 주식 종목을 리스트 형태로 제공합니다. 해당 메서드를 호출하기 위해 먼저 win32com.client 모듈을 임포트하고 CpCodeMgr 클래스의 객체를 생성합니다.

instCpCodeMgr 변수가 생성된 객체를 바인딩하고 있으므로 앞으로 instCpCodeMgr를 통해 GetStock ListByMarket 메서드를 호출하면 됩니다. 이때 CetStockListByMarket 메서드의 인자로 1을 전달하면 유가증권시장의 종목을 파이썬 튜플 형태로 반환받을 수 있습니다.

In [5]:
instCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")
codeList = instCpCodeMgr.GetStockListByMarket(1)

이번에는 반복문을 통해 각 종목의 종목명을 구하고, 종목 코드를 키(key)로, 종목 이름을 값(value)으로 사전에 추가해봅시다. 다음 코드를 보면 종목 코드로부터 종목명을 구할 때는 CpCodeMgr 클래스의 CodeToName 메서드를 사용했습니다. 참고로 CpStockCode 클래스에도 CodeToName 메서드가 있었 습니다.

In [10]:
kospi = {}
for code in codeList:
    name = instCpCodeMgr.CodeToName(code)
    kospi[code] = name
    
'''
{'A000020': '동화약품',
 'A000040': 'KR모터스',
 ...
와 같이 dictionary형태로 반환. 
'''

"\n{'A000020': '동화약품',\n 'A000040': 'KR모터스',\n ...\n와 같이 dictionary형태로 반환. \n"

GetStockListByMarket 메서드를 통해 구한 주식 코드 리스트에는 ETF(Exchange Traded Fund)와 ETN(Exchange Traded Note)도 포함돼 있습니다. 만약 ETF나 ETN 종목은 제외하고 순수하게 유가증권시장에 상장된 종목에 대한 코드를 구하려면 어떻게 해야 할까요?

CpCodeMgr 클래스를 살펴보면 GetStockSectionKind 메서드가 있는데, 이 메서드를 이용하면 어떤 종목이 주권인지 ETF인지 또는 ETN인지 확인할 수 있습니다.

다음은 유가증권시장의 종목에 대해 인덱스, 종목 코드, 부 구분 코드, 종목명을 출력하는 코드입니다. CpCodeMgr 클래스의 GetStockSectionKind 메서드에 인자로 종목 코드를 전달하면 부 구분 코드를 반환받을 수 있습니다. CybosPlus 도움말 파일의 CpCodeMgr 클래스를 참조하면 1번이 주권, 10번이 ETF, 17번이 ETN임을 알 수 있습니다.

In [12]:
for i, code in enumerate(codeList):
    secondCode = instCpCodeMgr.GetStockSectionKind(code)
    name = instCpCodeMgr.CodeToName(code)
    
    if i < 30:
        print(i, code, secondCode, name) # ETF, ETN은 17번대에 있다. 

0 A000020 1 동화약품
1 A000040 1 KR모터스
2 A000050 1 경방
3 A000060 1 메리츠화재
4 A000070 1 삼양홀딩스
5 A000075 1 삼양홀딩스우
6 A000080 1 하이트진로
7 A000087 1 하이트진로2우B
8 A000100 1 유한양행
9 A000105 1 유한양행우
10 A000120 1 CJ대한통운
11 A000140 1 하이트진로홀딩스
12 A000145 1 하이트진로홀딩스우
13 A000150 1 두산
14 A000155 1 두산우
15 A000157 1 두산2우B
16 A000180 1 성창기업지주
17 A000210 1 대림산업
18 A000215 1 대림산업우
19 A000220 1 유유제약
20 A000225 1 유유제약1우
21 A000227 1 유유제약2우B
22 A000230 1 일동홀딩스
23 A000240 1 한국테크놀로지그룹
24 A000270 1 기아차
25 A000300 1 대유플러스
26 A000320 1 노루홀딩스
27 A000325 1 노루홀딩스우
28 A000370 1 한화손해보험
29 A000390 1 삼화페인트


## 2) 과거 데이터 구하기

알고리즘 트레이딩 시스템을 만들 때 중요한 일 중 하나가 바로 과거 데이터를 확보하는 것입니다. 예를 들어, 거래량 분석을 통해 유망한 투자 종목을 뽑는 프로그램을 개발한다면 먼저 각 종목에 대한 일정 기간의 거래량 데이터가 있어야 할 것입니다.

이러한 과거 데이터를 구하는 가장 효과적인 방법 중 하나가 바로 증권사에서 제공하는 API를 사용하는 것입니다. 이번 절에서는 API를 이용해 주식 종목의 시가, 고가, 저가, 종가, 거래량 등의 데이터를 가져오는 방법을 배우겠습니다.

대신증권의 CYBOS Plus 매뉴얼을 참조하면 과거 데이터를 조회하는 데 StockChart 클래스를 사용할 수 있음을 알 수 있습니다. 참고로 StockChart 클래스는 그림 10.18과 같이 CpSysDib 모듈에 위치 합니다.

먼저 API 사용법을 알아보기 위해 대신증권 종목에 대한 최근 10일간 종가 데이터를 구하는 코드를 작성해보겠습니다.

CYBOS Plus의 StockChar 클래스를 사용하려면 먼저 StockChart 클래스의 인스턴스를 생성해야 합니다.

In [52]:
instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")

StockChart 클래스는 앞에서 배운 클래스에 비해 사용하기가 조금 복잡합니다. 그림 10.18에 있는 StockChart 클래스의 설명을 참조하면 통신 종류가 Request/Reply임을 알 수 있습니다. 여기서 Request/Reply 방식이란 사용자가 어떤 정보를 얻고자 할 때 얻으려는 정보가 무엇인지를 정확히 알려준 후 요청(Request)하면 CYBOS Plus가 이에 대한 답을 돌려주는(Reply) 방식을 의미합니다.

계속해서 코드를 완성해 보겠습니다. 앞서 instStockChart 객체를 생성했으므로 해당 객체를 통해 메서드를 호출하면 됩니다. 맨 먼저 할 일은 어떤 데이터를 원하는지 정확히 알려주는 일입니다. 이때 사용하는 메서드가 바로 SetInputValue입니다.

In [53]:
# 종목과 조회조건 등을 설정. 

instStockChart.SetInputValue(0, "A003540") # 첫 인자는 데이터 타입, 두 번째 인자는 입력 데이터 값. 0은 종목코드, A003540은 종목코드. 
instStockChart.SetInputValue(1, ord('2')) # 첫 번째 인자는 기간/요청 설정을 말하는 1. 두 번째 인자는 기가능로 요청 시 1, 개수로 요청시 2. 
instStockChart.SetInputValue(4, 10) # 첫 번째 인자는 요청개수 설정을 말하는 4. 두 번째 인자는 실제로 몇 개 요청할 것인지 개수. (여기선 10개)
instStockChart.SetInputValue(5, 5) # 첫 번째 인자는 요청할 데이터의 종류를 말하는 5. 두 번째 인자는 어떤 인자를 요청할 지. 5는 종가. 
instStockChart.SetInputValue(6, ord('D')) # 첫 번째 인자는 차트의 종류를 말하는 6. 두 번째 인자는 어떤 차트인지. ord('D')는 일단위
instStockChart.SetInputValue(9, ord('1')) # 첫 번째 인자는 수정주가의 반영여부를 말하는 9, 두 번째 인자는 수정주가를 의미하는 ord('1')

코드를 보면 총 6번에 걸쳐 SetInputValue 메서드를 호출했습니다. SetInputValue 메서드의 첫 번째 인자는 입력 데이터 타입을 의미하고 두 번째 인자는 입력 데이터 값을 의미합니다.

예를 들어, SetInputValue(0, "A003540")에서 0은 종목 코드를 의미하고, 'A003540'은 조회하려는 종목의 코드값입니다. 앞서 대신증권 종목의 최근 10일 종가를 확인하기로 했는데 대신증권의 종목 코드가 바로 'A003540'입니다.

조회할 종목 코드에 대한 정보를 입력했으면 다음으로 조회할 기간을 입력합니다. StockChart 클래스는 종목에 대한 과거 데이터를 기간 또는 개수로 조회할 수 있습니다. 예제에서는 최근 10일간의 종가만 얻기 위해 개수로 지정했습니다. 기간으로 요청할 때는 '1'을 입력하고, 개수로 요청할 때는 '2'를 입력합니다. 다만 파이썬의 문자열을 그대로 입력하는 것이 아니라 ord라는 함수를 통해 문자열 값을 아스키 코드로 변환해야 합니다.

세 번째로 입력한 데이터는 요청 개수입니다. SetInputValue(4, 10)에서 4는 요청 개수라는 타입을 의미하고 10이 실제로 요청할 데이터의 개수를 의미합니다. 10은 최근 거래일로부터 10일 치에 해당하는 데이터를 의미합니다.

네 번째로 입력한 데이터는 요청할 데이터의 종류입니다. CYBOS Plus 도움말에 있는 StockChart 부분을 참조하면 종가에 해당하는 값이 5임을 알 수 있습니다.

다섯 번째로 입력한 데이터는 차트의 종류로서 일 단위의 데이터를 가져오기 위해 ord('D')를 입력했습니다. 여섯 번째로 입력한 데이터는 수정 주가의 반영 여부에 대한 것으로 수정 주가를 의미하는 ord('1')을 입력했습니다.

이처럼 조회하려는 종목과 조회 조건에 대한 정보를 SetInputValue 메서드를 통해 입력하고 나면 데이터 처리를 요청하면 됩니다.

In [59]:
# instStockChart.BlockRequest() # 데이터 처리 요청. # 해당 요청이 없으면 GetHeaderValue 나 GetDataValue 등을 처리할 수 없다. 받아온게 없으니까. 

Request/Reply 방식에서는 사용자가 BlockRequest 메서드 호출을 통해 서버에 데이터를 요청(request)하면 서버는 해당 데이터를 준비한 후 사용자에게 응답(reply)합니다. 서버로부터 데이터를 얻어올 때는 GetHeaderValue와 GetDataValue라는 메서드를 사용합니다.

먼저 GetHeaderValue 메서드를 통해 수신한 데이터의 개수를 확인합니다. 앞서 최근 10일 치에 대한 데이터를 요청했으므로 아마도 numData의 값은 10일 것입니다.

In [55]:
numData = instStockChart.GetHeaderValue(3)
print(type(numData))
numData

<class 'int'>


0

numData만큼의 데이터를 가져오기 위해 간단히 for 문을 작성하고 해당 for 문 내에서 GetDataValue 메서드를 호출해 10일 치 데이터를 화면에 출력하면 됩니다.

In [56]:
for i in range(numData):
    print(i)
    print(instStockChart.GetDataValue(0, i)) # 첫 인자는 요청한 데이터의 인덱스(column #). 두 번째 인자는 row #

GetDataValue 메서드의 첫 번째 인자는 요청한 데이터의 인덱스를 의미합니다. 앞서 일 단위로 종가 하나만 요청했으므로 이 값으로 0을 입력하면 됩니다. 만약 일 단위로 시가, 고가, 저가, 종가, 거래량을 얻어오는 경우라면 시가를 얻을 때는 0, 고가를 얻을 때는 1을 사용하면 됩니다. GetDataValue 메서드의 두 번째 인자는 for 문의 인덱스 값을 그대로 사용하면 됩니다.

그림 10.20은 CYBOS Plus의 Request/Reply 타입의 객체를 사용하는 절차를 그림으로 나타낸 것입니다. 맨 먼저 win32com.client 모듈의 Dispatch 함수를 사용해 객체를 생성합니다. 그런 다음 생성된 객체의 SetInputValue 메서드를 호출해 요청할 데이터에 대한 정보를 입력합니다. 데이터 입력이 끝나면 서버에 데이터 처리를 요청하는 BlockRequest 메서드를 호출합니다. 서버로부터 데이터 처리에 대한 응답이 오면 GetDataValue 메서드를 이용해 데이터를 가져옵니다.

CYBOS Plus에는 수많은 클래스가 있는데, 그중 많은 클래스가 Request/Reply 방식으로 동작합니다. 따라서 그림 10.20의 절차를 잘 기억해 둔다면 Request/Reply 방식의 다른 클래스도 손쉽게 사용할 수 있을 것입니다.

지금까지 StockChart 클래스의 기본적인 사용법을 익혔습니다. 이번에는 조금 더 프로그램을 확장해 보겠습니다. 먼저 대신증권 종목에 대해 일자별로 시가, 고가, 저가, 종가, 거래량을 가져오는 프로그램을 구현해 보겠습니다.

In [57]:
instStockChart.SetInputValue(5, [0, 2, 3, 4, 5, 8]) # 첫 인자는 필드 설정에 관련된 5, 두 번째 인자는 가져올 column들. 튜플또는 리스트로 로 여러 개 입력 가능. 

instStockChart.BlockRequest() # 데이터 처리 요청. 

numData = instStockChart.GetHeaderValue(3)
numField = instStockChart.GetHeaderValue(1)



print(numData)
print(numField)

10
6


In [58]:
for i in range(numData):
    for j in range(numField):
        print(instStockChart.GetDataValue(j,i), end=' ')
    print(" ")

20200103 11950 12000 11800 11950 88954  
20200102 11850 12000 11850 11950 94480  
20191230 11950 11950 11800 11950 89190  
20191227 11950 12100 11800 11950 213542  
20191226 12350 12400 12200 12350 260656  
20191224 12450 12500 12300 12300 99029  
20191223 12500 12500 12300 12450 112315  
20191220 12500 12500 12350 12450 43130  
20191219 12250 12450 12150 12450 135315  
20191218 12150 12200 12100 12200 97194  


기타 기간 등 조건 바꿔 가능하다. 

이는 항상 api doc을 참고하여 이뤄져야 한다. 

CpSysDib.StockChart 
http://money2.daishin.com/e5/mboard/ptype_basic/HTS_Plus_Helper/DW_Basic_Read_Page.aspx?boardseq=284&seq=102&page=1&searchString=stockchart&p=&v=&m=

## 3) PER, EPS 데이터 구하기

이번 절에서는 종목별로 PER과 EPS 데이터를 구하는 프로그램을 작성해보겠습니다. 먼저 CYBOS Plus 도움말 파일의 검색 탭을 이용해 ‘per’이라는 키워드로 검색하면 그림 10.21과 같이 MarketEye 클래스에서 PER을 지원하는 것을 확인할 수 있습니다.

MarketEye 클래스의 통신 종류를 살펴보면 역시 Request/Reply 방식임을 확인할 수 있습니다. 따라서 그림 10.20과 같은 방식으로 MarketEye 클래스를 사용하면 됩니다. 먼저 MarketEye 클래스에 대한 인스턴스를 생성합니다.

In [60]:
instMarketEye = win32com.client.Dispatch("CpSysDib.MarketEye")

SetInputValue 메서드를 이용해 타입 0에 요청하고자 하는 필드 값을 설정합니다. 예를 들어, 현재가, PER, EPS, 최근분기년월 데이터를 요청하려면 (4, 67, 70, 111)을 입력하면 됩니다. 더 자세한 내용은 CYBOS Plus 도움말 파일을 참조하기 바랍니다.

타입 1에는 조회할 종목에 대한 종목 코드를 지정하면 됩니다. 따라서 대신증권 종목에 대해 현재가, PER, EPS, 최근분기년월을 구하려면 다음과 같이 코드를 작성하면 됩니다.

In [61]:
instMarketEye.SetInputValue(0, (4, 67, 70, 111)) # current price, PER, EPS, 최근분기년월
instMarketEye.SetInputValue(1, 'A003540') # 대신증권

instMarketEye.BlockRequest()

0

마지막으로 GetDataValue 메서드를 이용해 서버로부터 반환된 데이터를 출력하면 됩니다. 이때 GetHeaderValue 메서드를 통해 반환되는 필드의 개수나 종목 개수를 확인할 수도 있지만 이미 알고 있다면 바로 GetDataValue 메서드를 사용해도 됩니다.

GetDataValue 메서드에서 첫 번째 인자는 필드에 대한 인덱스이고 두 번째 인자는 종목의 인덱스입니다. 지금은 대신증권 종목 하나에 대한 데이터를 요청했으므로 두 번째 인자의 값은 0이 됩니다.

앞서 필드 항목으로 현재가, PER, EPS, 최근분기연월을 요청했으므로 첫 번째 인자로 각각 0, 1, 2, 3을 지정하면 됩니다.

In [62]:
print("현재가: ", instMarketEye.GetDataValue(0, 0))
print("PER: ", instMarketEye.GetDataValue(1, 0))
print("EPS: ", instMarketEye.GetDataValue(2, 0))
print("최근분기년월: ", instMarketEye.GetDataValue(3, 0))

현재가:  11950
PER:  12.15999984741211
EPS:  982
최근분기년월:  201909
